# Shore Line Feature Detection Worksheet

Data available:
- .txt file containing longitude, latitude, height, red, green, blue values
- several .tif orthophotos

With no labeled data our goals are:
- provide an unsupervised clustering algorithm to extract distinct features from .txt file including the above data
- give suggestions on how this feature extraction algorithm can be used to label data with human intervention
- reduce the need for humans to trace out boundaries of distinct features

## Data Management (scalable if system has more storage and memory)

First step is to import the relevant python packages

In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn import svm
import seaborn as sns

import scipy as sp
import scipy.sparse as sps

%matplotlib inline

Next we select a random subsample of the original data containing 10,000,000 data points (roughly $1/6^{\text{th}}$ of the total)

In [ ]:
# obtain a shuffled data set from the original text file using bash commands
!shuf -n 10000000 /home/divgrad/data/4-Vadeboncoeur/davis-bay.txt > /home/divgrad/bcdata-project-temp/sample-temp.txt
data = pd.read_csv('/home/divgrad/bcdata-project-temp/sample-temp.txt',sep=" ", header=None)
!rm /home/divgrad/bcdata-project-temp/sample-temp.txt

Each data poitn is represented by a row and each columnb represents: longitude, latitude, heigh [m], red, green, blue.
The last three rows are not used (they likely represent some sort of angle and are not necessary for this analysis.

In [ ]:
data.columns = ['lon', 'lat', 'z', 'r', 'g', 'b', '?', '?', '?']

# extract height data from pandas dataframe
lon, lat = data['lon'].values, data['lat'].values
z = data['z'].values
rgb = data[['r','g','b']].values

## Level One: Clustering With Respect to Height Variation

Since the data list is unordered, we instead use a nearest neighbour algorithm to determine which points are neighbours of each other. This is stored in a connectivity matrix, A, from which we can now extract useful information such as the average height in each neighbourhood as well as the standard deviation.

In [ ]:
# we work only with the first million data entries (computation on all 10,000,000 too computationally intensive)
lon_1, lat_1 = lon[:1000000], lat[:1000000]
z = z[:1000000]
rgb_1 = rgb[:1000000,:]

# radius around each point in which we will be calculating height variation
neighbor_radius = np.sqrt((lon_1.max()-lon_1.min())**2 + (lat_1.max()-lat_1.min())**2)/5000

# for each point find all neighbors within specified distance away
nn = NearestNeighbors(radius=neighbor_radius,metric='euclidean')
nn.fit(np.column_stack((lon_1,lat_1)))

# connectivity matrix: 0 = not a neighbor | 1 = a neighbor
A = nn.radius_neighbors_graph()

# this vector and diagonal matrices give you 1/(number of neighbourrs including self)
q = 1/(np.squeeze(np.asarray(A.sum(axis=1)))+1)
Q = sps.diags(q)
A_ave = Q.dot(A) + sps.eye(Q.shape[0])

# matrix multiplication with z gives average height of neighbors for each point
z_ave = A_ave.dot(z)

# compute the standrad deviation of all neighbors for each point (including self)
z_std = np.sqrt(A_ave.dot((z-z_ave)**2))

Now we use mini-batch k-means to go through the data and perform the k-means clustering algorithm. Our choice of 5 clusters was found through visual trial and error as well as comparison of color and heigh distributions. More work can (and should) be done in the future to determine optimal number of clusters. An ideal system would continue to increase the number of clusters until some stopping criteria is met. Some suggestions for future work include:
- developing a quantitative way to measure differences in spatial variation among clusters, and stop increasing the number of clusters when this difference is sufficiently small.
- research regions where algorithm will be applied to determine characteristics of the land and choose the number of clusters before hand. Then compare the clusters with the expected characteristics of the land.

In [ ]:
batch_size = 50000
num_clusters = 5

# create mini batch k-means object
mbk_zstd = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)

# fit the 1,000,000 standard deviation in height data with the k-means model
mbk_zstd.fit(z_std.reshape(-1,1));

# extract the labels: 0, 1, 2, 3, 4
mbk_zstd_labels = mbk_zstd.labels_

In [ ]:
mbk_zstd.cluster_centers_

In [ ]:
# the index sampling in this piece of code is for visualization purposes only; we have to avoid plotting 1,000,000 points!
smpl_idx_1 = np.random.choice(np.arange(len(lon_1)), 150000, replace=False)

smpl_lon_1, smpl_lat_1  = lon[smpl_idx_1], lat[smpl_idx_1]
smpl_z_1 = z[smpl_idx_1]
smpl_rgb_1 = rgb[smpl_idx_1]
smpl_rgb_1_scaled = smpl_rgb_1/255
smpl_labels_1 = mbk_zstd_labels[smpl_idx_1]


## Visualization of Height Standard Deviation Clustering Results

### Original Image

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1,smpl_lat_1,c=smpl_rgb_1_scaled,s=3,lw=0);
plt.axis('scaled')
plt.show()

### Cluster Color Coded

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1,smpl_lat_1,c=smpl_labels_1,s=3,lw=0,cmap='plasma');
plt.axis('scaled')
plt.show()

### Cluster Label 0

In [ ]:
curr_label = 0

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1[smpl_labels_1==curr_label],smpl_lat_1[smpl_labels_1==curr_label],c=smpl_rgb_1_scaled[smpl_labels_1==curr_label,:],s=3,lw=0);
plt.axis('scaled')
plt.show()

# Cluster Label 1

In [ ]:
curr_label = 1

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1[smpl_labels_1==curr_label],smpl_lat_1[smpl_labels_1==curr_label],c=smpl_rgb_1_scaled[smpl_labels_1==curr_label,:],s=3,lw=0);
plt.axis('scaled')
plt.show()

### Cluster Label 2

In [ ]:
curr_label = 2

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1[smpl_labels_1==curr_label],smpl_lat_1[smpl_labels_1==curr_label],c=smpl_rgb_1_scaled[smpl_labels_1==curr_label,:],s=3,lw=0);
plt.axis('scaled')
plt.show()

### Cluster Label 3

In [ ]:
curr_label = 3

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1[smpl_labels_1==curr_label],smpl_lat_1[smpl_labels_1==curr_label],c=smpl_rgb_1_scaled[smpl_labels_1==curr_label,:],s=3,lw=0);
plt.axis('scaled')
plt.show()

### Cluster Label 4

In [ ]:
curr_label = 4

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon_1[smpl_labels_1==curr_label],smpl_lat_1[smpl_labels_1==curr_label],c=smpl_rgb_1_scaled[smpl_labels_1==curr_label,:],s=3,lw=0);
plt.axis('scaled')
plt.show()

## Visualizing Height and Colour (RGB) Variability in Each Cluster

<b>Height Variability</b>

In [ ]:
plt.figure(figsize=(15,9))

for curr_label in range(num_clusters):
    plt.subplot(num_clusters,1,curr_label+1)
    plt.hist(z[mbk_zstd_labels==curr_label],bins='auto',color='0.5',lw=0,normed=True)
    plt.xlim((np.min(z),np.max(z)))
    

plt.show()

The histograms for clusters 1, 2, and 3 suggest further clustering could be done. This is where a cluster number selection algorithm would be most useful.

<b>RGB Variability</b>

In [ ]:
plt.figure(figsize=(15,9))

for curr_label in range(num_clusters):
    plt.subplot(num_clusters,3,3*curr_label+1)
    plt.hist(rgb_1[mbk_zstd_labels==curr_label,0],bins=255,color='r',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+2)
    plt.hist(rgb_1[mbk_zstd_labels==curr_label,1],bins=255,color='g',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+3)
    plt.hist(rgb_1[mbk_zstd_labels==curr_label,2],bins=255,color='b',lw=0,normed=True)
    plt.xlim((0,255))

plt.show()

The RGB histograms indicate that there is potential for a lot of clustering with respect to RGB values within each level one cluster.

# Level Two: RGB Sub-Clustering

In [ ]:
def variation_color(r):
    suma = r[:,0]+r[:,1]+r[:,2]
    vector1 = r[:,0]/suma
    vector2 = r[:,1]/suma
    vector3 = r[:,2]/suma
    
    return np.hstack([[vector1,vector2,vector3]]).T

In [ ]:
neigh = KNeighborsClassifier(radius=1e-5)
neigh.fit(np.column_stack((lon_1, lat_1)),mbk_zstd_labels);

space = np.column_stack((lon,lat))
labels = neigh.predict(space)
data1 = np.column_stack((lon,lat,rgb))

data1_0 = np.array([j for i,j in enumerate(data1) if labels[i] == 0])
cluster_0 = data1_0[:,[2,3,4]]
lon_0 = data1_0[:,0]
lat_0 = data1_0[:,1]
data1_1 = np.array([m for l,m in enumerate(data1) if labels[l] == 1])
cluster_1 = data1_1[:,[2,3,4]]
lon_1 = data1_1[:,0]
lat_1 = data1_1[:,1]
data1_2 = np.array([r for s,r in enumerate(data1) if labels[s] == 2])
cluster_2 = data1_2[:,[2,3,4]]
lon_2 = data1_2[:,0]
lat_2 = data1_2[:,1]
data1_3 = np.array([u for w,u in enumerate(data1) if labels[w] == 3])
cluster_3 = data1_3[:,[2,3,4]]
lon_3 = data1_3[:,0]
lat_3 = data1_3[:,1]
data1_4 = np.array([p for q,p in enumerate(data1) if labels[q] == 4])
cluster_4 = data1_4[:,[2,3,4]]
lon_4 = data1_4[:,0]
lat_4 = data1_4[:,1]

## Cluster 0 - Subclustering

In [ ]:
batch_size = 50000
num_clusters = 5

mbk_rgb0 = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)

cluster_00 = variation_color(np.array(cluster_0));

mbk_rgb0.fit(np.array(cluster_00))
mbk_labels = mbk_rgb0.labels_

In [ ]:
smpl_idx = np.random.choice(np.arange(len(cluster_0)), 150000, replace=False)
smpl_lon, smpl_lat, smpl_rgb  = np.array(lon_0)[smpl_idx], np.array(lat_0)[smpl_idx], np.array(cluster_0)[smpl_idx]
smpl_labels = mbk_labels[smpl_idx]

cluster_01 = np.array(cluster_0)/255
smpl_rgb = np.array(cluster_0)[smpl_idx]
smpl_rgb_01 = cluster_01[smpl_idx]

<b> Cluster 0 (Original) </b>

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(smpl_lon,smpl_lat,c=smpl_rgb_01,s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 0</b>

In [ ]:
curr_label = 0

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_01[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 1</b>

In [ ]:
curr_label = 1

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_01[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 2</b>

In [ ]:
curr_label = 2

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_01[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 3</b>

In [ ]:
curr_label = 3

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_01[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 4</b>

In [ ]:
curr_label = 4

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_01[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

### Color distribution for Cluster 0 Subclustering

In [ ]:
curr_label = 0

plt.figure(figsize=(15,9))

for curr_label in range(num_clusters):
    plt.subplot(num_clusters,3,3*curr_label+1)
    plt.hist(smpl_rgb[smpl_labels==curr_label,0],bins=255,color='r',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+2)
    plt.hist(smpl_rgb[smpl_labels==curr_label,1],bins=255,color='g',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+3)
    plt.hist(smpl_rgb[smpl_labels==curr_label,2],bins=255,color='b',lw=0,normed=True)
    plt.xlim((0,255))

plt.show()

## Cluster 1 - Subclustering

In [ ]:
batch_size = 50000
num_clusters = 5

mbk_rgb1 = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)

cluster_11 = variation_color(np.array(cluster_1));

mbk_rgb1.fit(np.array(cluster_00))
mbk_labels = mbk_rgb1.labels_

In [ ]:
smpl_idx = np.random.choice(np.arange(len(cluster_1)), 150000, replace=False)
smpl_lon, smpl_lat, smpl_rgb  = np.array(lon_1)[smpl_idx], np.array(lat_1)[smpl_idx], np.array(cluster_1)[smpl_idx]
smpl_labels = mbk_labels[smpl_idx]

cluster_11 = np.array(cluster_1)/255
smpl_rgb = np.array(cluster_0)[smpl_idx]
smpl_rgb_11 = cluster_11[smpl_idx]

<b> Cluster 0 (Original) </b>

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(smpl_lon,smpl_lat,c=smpl_rgb_11,s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 0</b>

In [ ]:
curr_label = 0

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_11[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 1</b>

In [ ]:
curr_label = 1

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_11[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 2</b>

In [ ]:
curr_label = 2

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_11[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 3</b>

In [ ]:
curr_label = 3

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_11[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()

<b> Cluster 0 - 4</b>

In [ ]:
curr_label = 4

plt.figure(figsize=(15,15))
plt.scatter(smpl_lon[smpl_labels==curr_label],smpl_lat[smpl_labels==curr_label],c=smpl_rgb_11[smpl_labels==curr_label,:],s=3,lw=0)
plt.axis('scaled')
plt.show()


### Color distribution for Cluster 0 Subclustering

In [ ]:
curr_label = 0

plt.figure(figsize=(15,9))

for curr_label in range(num_clusters):
    plt.subplot(num_clusters,3,3*curr_label+1)
    plt.hist(smpl_rgb[smpl_labels==curr_label,0],bins=255,color='r',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+2)
    plt.hist(smpl_rgb[smpl_labels==curr_label,1],bins=255,color='g',lw=0,normed=True)
    plt.xlim((0,255))
    plt.subplot(num_clusters,3,3*curr_label+3)
    plt.hist(smpl_rgb[smpl_labels==curr_label,2],bins=255,color='b',lw=0,normed=True)
    plt.xlim((0,255))

plt.show()